## Part V: Training a Model with Data Programming

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 2min 38s, sys: 12.7 s, total: 2min 51s
Wall time: 2min 57s


In [6]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels')

CPU times: user 2.24 s, sys: 683 ms, total: 2.92 s
Wall time: 3.03 s


## Train Generative Model

In [7]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
# gen_model.train(L_train, n_iter=3000, rate=1e-5)
gen_model.train(L_train, n_iter=15000, rate=1e-2)

Training marginals (!= 0.5):	75908
Features:			20
Begin training for rate=0.01, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.630145
	Learning epoch = 250	Gradient mag. = 1.155185
	Learning epoch = 500	Gradient mag. = 1.610080
	Learning epoch = 750	Gradient mag. = 1.242761
	Learning epoch = 1000	Gradient mag. = 0.977723
	Learning epoch = 1250	Gradient mag. = 0.846643
	Learning epoch = 1500	Gradient mag. = 0.602580
	Learning epoch = 1750	Gradient mag. = 0.433504
	Learning epoch = 2000	Gradient mag. = 0.345236
	Learning epoch = 2250	Gradient mag. = 0.291882
	Learning epoch = 2500	Gradient mag. = 0.255884
	Learning epoch = 2750	Gradient mag. = 0.230345
	Learning epoch = 3000	Gradient mag. = 0.212040
	Learning epoch = 3250	Gradient mag. = 0.199078
	Learning epoch = 3500	Gradient mag. = 0.190156
	Learning epoch = 3750	Gradient mag. = 0.184311
	Learning epoch = 4000	Gradient mag. = 0.180815
	Learning epoch = 4250	Gradient mag. = 0.179120
	Learning epoch = 4500	Gradient mag. = 0.178814
	Lear

In [8]:
gen_model.save(session, 'Generative Params')

In [9]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [10]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=4500, rate=1e-3)

Training marginals (!= 0.5):	31893
Features:			19168
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 22106.543030	Gradient magnitude = 60040.581956
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 34359.616881	Gradient magnitude = 4497.059732
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 1944.161946	Gradient magnitude = 2019.485449
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 19082.007108	Gradient magnitude = 11676.621825
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 24453.248688	Gradient magnitude = 4499.029256
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 7762.432351	Gradient magnitude = 11941.938575
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 23932.230869	Gradient magnitude = 4500.803038
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 9930.242639	Gradient magnitude = 9781.491858
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 3782.694326	Gradient magnitu

In [11]:
disc_model.w.shape

(19168,)

In [12]:
%time disc_model.save(session, "Discriminative Params")

CPU times: user 3.92 s, sys: 97.2 ms, total: 4.01 s
Wall time: 4.05 s


## Assess Performance on Development Set

In [13]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Loading the Development Feature Matrix

In [14]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_dev = feature_manager.load(session, dev, 'Train Features')

CPU times: user 1min 38s, sys: 14.8 s, total: 1min 53s
Wall time: 2min 4s


In [15]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Silver').one()

In [16]:
# from snorkel.loaders import create_or_fetch
# from snorkel.models import Label, AnnotationKeySet
# annotation_key_name = "Hardware Development Labels - Gold"
# annotation_key = create_or_fetch(session, AnnotationKey, annotation_key_name)
# session.query(Label).filter(Label.key == annotation_key).count()

In [17]:
from snorkel.models import CandidateSet
from snorkel.annotations import LabelManager
label_manager = LabelManager()
# from snorkel.utils import get_ORM_instance
# dev_labels = get_ORM_instance(AnnotationKeySet, session, 'Hardware Development Labels - Gold')
L_dev = label_manager.load(session, dev, 'Hardware Development Labels -- Silver')

In [18]:
gold_dev_set = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Silver').one()

In [19]:
F_dev.shape

(18971, 19168)

In [20]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

Calibration plot:
Recall-corrected Noise-aware Model
Pos. class accuracy: 0.698890010091
Neg. class accuracy: 0.757311555016
Corpus Precision 0.759
Corpus Recall    0.699
Corpus F1        0.728
----------------------------------------
TP: 6926 | FP: 2199 | TN: 6862 | FN: 2984

Recall-corrected Noise-aware Model
Pos. class accuracy: 0.698890010091
Neg. class accuracy: 0.757311555016
Corpus Precision 0.759
Corpus Recall    0.699
Corpus F1        0.728
----------------------------------------
TP: 6926 | FP: 2199 | TN: 6862 | FN: 2984



/Users/bradenhancock/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [21]:
from hardware_utils import part_error_analysis, table_info
#     print c[0]
#     print c[0].parent
#     print (c[0].parent.row_num, c[0].parent.col_num)
#     print c[1]
#     print c[1].parent
#     print (c[1].parent.row_num, c[1].parent.col_num)

In [22]:
# for c in list(fp)[:5]:
#     print c
# c = list(fp)[10]
# error_analysis(c)
# c = list(fp)[10]
# part_error_analysis(c)
# print c.temp.parent.row_num

# for c in list(fp)[:5]:
#     print c
# print c.temp.parent.html_anc_tags

In [23]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 0.822
Corpus Recall    1.0
Corpus F1        0.902
----------------------------------------
TP: 152 | FP: 33 | FN: 0



In [24]:
print FN

set([])


In [25]:
from pprint import pprint
pprint(FP)

set([(u'ONSMS04099-1', u'BC237', u'-65'),
     (u'ONSMS04099-1', u'BC237A', u'-65'),
     (u'ONSMS04099-1', u'BC237B', u'-65'),
     (u'ONSMS04099-1', u'BC237C', u'-65'),
     (u'ONSMS04099-1', u'BC238', u'-65'),
     (u'ONSMS04099-1', u'BC238B', u'-65'),
     (u'ONSMS04099-1', u'BC238C', u'-65'),
     (u'ONSMS04099-1', u'BC239', u'-65'),
     (u'ONSMS04099-1', u'BC239C', u'-65'),
     (u'PHGLS19500-1', u'MMBT3904', u'-55'),
     (u'PNJIS01593-1', u'BC856', u'-65'),
     (u'PNJIS01593-1', u'BC856A', u'-65'),
     (u'PNJIS01593-1', u'BC856B', u'-65'),
     (u'PNJIS01593-1', u'BC857', u'-65'),
     (u'PNJIS01593-1', u'BC857A', u'-65'),
     (u'PNJIS01593-1', u'BC857B', u'-65'),
     (u'PNJIS01593-1', u'BC857C', u'-65'),
     (u'PNJIS01593-1', u'BC858', u'-65'),
     (u'PNJIS01593-1', u'BC858A', u'-65'),
     (u'PNJIS01593-1', u'BC858B', u'-65'),
     (u'PNJIS01593-1', u'BC858C', u'-65'),
     (u'PNJIS01593-1', u'BC859', u'-65'),
     (u'PNJIS01593-1', u'BC859B', u'-65'),
     (u'PNJIS015

In [26]:
import os
os.system('cp snorkel.db snorkel.db\ final');

In [27]:
from pprint import pprint
all_weights = disc_model.get_feature_weights(F_train)

In [28]:
pprint(all_weights[:60])

[(AnnotationKey (TAB_e2_ROW_LEMMAS_[temperature]), 12.648358601523558),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[t]), 12.445072897164422),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[storage]), 11.125439293776846),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[stg]), 9.327678702656604),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[t stg]), 9.326679759799017),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[storage temperature]), 9.241757295165732),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[% c]), 7.406441630200583),
 (AnnotationKey (TAB_e2_COL_LEMMAS_[65]), 7.244077268670047),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[temperature range]), 6.081553911728434),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[range]), 6.081553911728434),
 (AnnotationKey (TAB_e2_ROW_LEMMAS_[c]), 5.727914212488628),
 (AnnotationKey (TAB_e2_W_RIGHT_POS_1_[CD]), 4.863244305381174),
 (AnnotationKey (TAB_e2_W_POS_L_1_R_1_[CD]_[CD]), 4.863244305381174),
 (AnnotationKey (TAB_e2_W_RIGHT_1_[_NUMBER]), 4.821288705332301),
 (AnnotationKey (TAB_e2_W_LEMMA_L_1_R_1_[_NUMBER]_[_NUMBER]),


In [29]:
fp_zetx = set()
print(len(fp))
for f in fp:
    if str(f.part.parent.document) == "Document ZETXS01948-1":
        fp_zetx.add(f)

print(len(fp_zetx))

2199
0


In [30]:
sample = fp_zetx.pop()

KeyError: 'pop from an empty set'

In [ ]:
from snorkel.utils import get_keys_by_candidate
for a in sorted(get_keys_by_candidate(F_dev, sample)):
    print a

In [ ]:
from snorkel.lf_helpers import *
def LF_v_row(c):
    return -1 if 'v' in get_row_ngrams(c.temp, infer=True) else 0

LF_v_row(sample)

The End.